<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/Wikipedia_Events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streaming Wikipedia Events

FROM: https://github.com/donaghhorgan/COMP9033/blob/master/lectures/12%20-%20Streaming%20Data%20Analysis.pdf



In [1]:
#install software
# No clustering for this install !!
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 44.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=757f23a7a89981eadd48a67a385f97d7807ef05ff5033f7d028f072ddda947f8
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
# start the spark session

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("Wikipedia Event") \
    .getOrCreate()

### Streaming Wikipedia events

Currently, Spark supports three kinds of streaming connection out of the box:

FROM:
https://github.com/donaghhorgan/COMP9033/blob/master/lectures/12%20-%20Streaming%20Data%20Analysis.pdf 

1. Connect to the Wikipedia RecentChanges stream using SSEClient.
2. Create a local socket connection on port 50000.
3. When a client (e.g. Spark) connects to the local socket, relay the next available event to it from the event stream.

In [3]:
!pip install sseclient
from pyspark.streaming import StreamingContext
from sseclient import SSEClient
import threading

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5584 sha256=2d16b107633552a218102c582b6500632e90bee6345099a759639d9d4815cc15
  Stored in directory: /root/.cache/pip/wheels/07/67/7e/96edf627ac746de1a5c5cbb8d59ed960f033b8352dc12c545d
Successfully built sseclient


In [ ]:
events = SSEClient('https://stream.wikimedia.org/v2/stream/recentchange')
for e in events:
  print(e)


{"$schema":"/mediawiki/recentchange/1.0.0","meta":{"uri":"https://www.wikidata.org/wiki/Q114775139","request_id":"31f7d9b1-12c4-4de4-bee7-fbdd8c055c4f","id":"6580fd51-0bde-4f48-8186-8b1f202c4e14","dt":"2022-10-20T13:20:31Z","domain":"www.wikidata.org","stream":"mediawiki.recentchange","topic":"eqiad.mediawiki.recentchange","partition":0,"offset":4248495942},"id":1809138679,"type":"edit","namespace":0,"title":"Q114775139","comment":"/* wbsetclaim-create:2||1 */ [[Property:P6634]]: ellen-rudnick-364a793","timestamp":1666272031,"user":"Korimako","bot":false,"minor":false,"patrolled":true,"length":{"old":2211,"new":2575},"revision":{"old":1754930719,"new":1754930819},"server_url":"https://www.wikidata.org","server_name":"www.wikidata.org","server_script_path":"/w","wiki":"wikidatawiki","parsedcomment":"‎<span dir=\"auto\"><span class=\"autocomment\">Created claim: </span></span> <a href=\"/wiki/Property:P6634\" title=\"Property:P6634\">Property:P6634</a>: ellen-rudnick-364a793"}
{"$schema

KeyboardInterrupt: ignored

In [127]:
import uuid
import json

def read_wikipedia():
    msgs = SSEClient('https://stream.wikimedia.org/v2/stream/recentchange')

    buffer=[]
    i=0
    for e in msgs:
      buffer.append(e)
      if len(buffer)>100:
        with open(f"streaming/{str(uuid.uuid4())}.json", "w") as outfile:
          json.dump([ob.__dict__ for ob in buffer], outfile)
        buffer.clear()
        i=i+1
        if i>3:
          return

threading.Thread(target=read_wikipedia).start()
#read_wikipedia()


In [85]:
import json

#don't forget the r for raw string...
data=r"""{"$schema":"/mediawiki/recentchange/1.0.0","meta":{"uri":"https://www.wikidata.org/wiki/Q114775139","request_id":"31f7d9b1-12c4-4de4-bee7-fbdd8c055c4f","id":"6580fd51-0bde-4f48-8186-8b1f202c4e14","dt":"2022-10-20T13:20:31Z","domain":"www.wikidata.org","stream":"mediawiki.recentchange","topic":"eqiad.mediawiki.recentchange","partition":0,"offset":4248495942},"id":1809138679,"type":"edit","namespace":0,"title":"Q114775139","comment":"/* wbsetclaim-create:2||1 */ [[Property:P6634]]: ellen-rudnick-364a793","timestamp":1666272031,"user":"Korimako","bot":false,"minor":false,"patrolled":true,"length":{"old":2211,"new":2575},"revision":{"old":1754930719,"new":1754930819},"server_url":"https://www.wikidata.org","server_name":"www.wikidata.org","server_script_path":"/w","wiki":"wikidatawiki","parsedcomment":"‎<span dir=\"auto\"><span class=\"autocomment\">Created claim: </span></span> <a href=\"/wiki/Property:P6634\" title=\"Property:P6634\">Property:P6634</a>: ellen-rudnick-364a793"}"""

#don't forget to build a list...
rdd=spark.sparkContext.parallelize([data])
df=spark.read.json(rdd)
df.show()

df.printSchema()
df.schema


+--------------------+-----+--------------------+----------+------------+--------------------+-----+---------+--------------------+---------+--------------------+----------------+------------------+--------------------+----------+----------+----+--------+------------+
|             $schema|  bot|             comment|        id|      length|                meta|minor|namespace|       parsedcomment|patrolled|            revision|     server_name|server_script_path|          server_url| timestamp|     title|type|    user|        wiki|
+--------------------+-----+--------------------+----------+------------+--------------------+-----+---------+--------------------+---------+--------------------+----------------+------------------+--------------------+----------+----------+----+--------+------------+
|/mediawiki/recent...|false|/* wbsetclaim-cre...|1809138679|{2575, 2211}|{www.wikidata.org...|false|        0|‎<span dir="auto"...|     true|{1754930819, 1754...|www.wikidata.org|              

StructType([StructField('$schema', StringType(), True), StructField('bot', BooleanType(), True), StructField('comment', StringType(), True), StructField('id', LongType(), True), StructField('length', StructType([StructField('new', LongType(), True), StructField('old', LongType(), True)]), True), StructField('meta', StructType([StructField('domain', StringType(), True), StructField('dt', StringType(), True), StructField('id', StringType(), True), StructField('offset', LongType(), True), StructField('partition', LongType(), True), StructField('request_id', StringType(), True), StructField('stream', StringType(), True), StructField('topic', StringType(), True), StructField('uri', StringType(), True)]), True), StructField('minor', BooleanType(), True), StructField('namespace', LongType(), True), StructField('parsedcomment', StringType(), True), StructField('patrolled', BooleanType(), True), StructField('revision', StructType([StructField('new', LongType(), True), StructField('old', LongTyp

## Streaming analysis

Now that we have our stream relay set up, we can start to analyse its contents. First, let's initialise a [`SparkContext`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.SparkContext) object, which will represent our connection to the Spark cluster. To do this, we must first specify the URL of the master node to connect to. As we're only running this notebook for demonstration purposes, we can just run the cluster locally, as follows:

Next, we create a [`StreamingContext`](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext) object, which represents the streaming functionality of our Spark cluster. When we create the context, we must specify a batch duration time (in seconds), to tell Spark how often it should process data from the stream. Let's process the Wikipedia data in batches of one second:

In [95]:
# stopping all active queries
# you will understand ;-)
for q in spark.streams.active:
  print("stopping",q.name)
  q.stop()
spark.sql(f"drop table wikipedia")

stopping wikipedia


DataFrame[]

In [96]:
#ssc = StreamingContext(sc, 1)
lines = spark \
    .readStream \
    .option("cleanSource","delete") \
    .format("json") \
    .schema(df.schema) \
    .load("streaming") \

results=lines.groupBy("user").count()

query = results \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .option("cleanSource","delete") \
    .queryName("wikipedia") \
    .start()

#query.awaitTermination()
print(query.isActive)

True


In [105]:
spark.sql("select * from wikipedia").show(3)
spark.sql("select count(*) from wikipedia").show()

!cp test.json streaming

+--------------------+-----+
|                user|count|
+--------------------+-----+
|               Kaitu|    2|
|2A00:1370:8190:88...|    1|
|            Renamerr|    1|
+--------------------+-----+
only showing top 3 rows

+--------+
|count(1)|
+--------+
|       4|
+--------+



Using our `StreamingContext` object, we can create a data stream from our local TCP relay socket with the [`socketTextStream`](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext.socketTextStream) method:

In [ ]:
stream = ssc.socketTextStream('localhost', 50000)

Even though we've created a data stream, nothing happens! Before Spark starts to consume the stream, we must first define one or more operations to perform on it. Let's count the number of edits made by different users in the last minute:

In [ ]:
users = (
    stream.map(json.loads)                   # Parse the stream data as JSON
          .map(lambda obj: obj['user'])      # Extract the values corresponding to the 'user' key
          .map(lambda user: (user, 1))       # Give each user a count of one
          .window(60)                        # Create a sliding window, sixty seconds in length
          .reduceByKey(lambda a, b: a + b)   # Reduce all key-value pairs in the window by adding values
          .transform(                        # Sort by the largest count
              lambda rdd: rdd.sortBy(lambda kv: kv[1], ascending=False))
          .pprint()                          # Print the results
)

Again, nothing happens! This is because the `StreamingContext` must be started before the stream is processed by Spark. We can start data streaming using the [`start`](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext.start) method of the `StreamingContext` and stop it using the [`stop`](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext.stop) method. Let's run the stream for two minutes (120 seconds) and then stop:

In [ ]:
ssc.start()

time.sleep(120)

ssc.stop()